Process a single recording. This involves:

- Define a configuration
- Read in the time data
- Preprocess the time data
- Decimate the time data
- Window the decimated data
- Calculate the spectra
- Calculate spectral values at evaluation frequencies
- Do the regression

In [19]:
from pathlib import Path
from resistics.config import Configuration
from resistics.time import TimeReaderNumpy
from resistics.gather import QuickGather
from resistics.decimate import Decimator

In [20]:
config = Configuration(name="testing")
config.decimator = Decimator(resample=False)

In [21]:
dir_path = Path("..", "data", "time", "quick", "kap123")
time_data = TimeReaderNumpy().run(dir_path)
ref_time = time_data.metadata.first_time

2023-04-28 08:06:36.895 | INFO     | resistics.time:run:777 - Reading time series data from ../data/time/quick/kap123
2023-04-28 08:06:36.896 | DEBUG    | resistics.time:run:779 - Reading time series metadata from ../data/time/quick/kap123
2023-04-28 08:06:36.898 | DEBUG    | resistics.time:run:787 - Reading samples from 0 to 361511
2023-04-28 08:06:36.901 | INFO     | resistics.time:read_data:1133 - Data successfully read from ../data/time/quick/kap123
2023-04-28 08:06:36.901 | DEBUG    | resistics.time:__init__:479 - Creating TimeData with data type float32
2023-04-28 08:06:36.902 | DEBUG    | resistics.time:run:790 - Applying scaling to time series data


In [22]:
for processor in config.time_processors:
    time_data = processor.run(time_data)

2023-04-28 08:06:36.932 | INFO     | resistics.time:run:1532 - Removing nan values from channels ['Hx', 'Hy', 'Hz', 'Ex', 'Ey']
2023-04-28 08:06:36.945 | DEBUG    | resistics.time:__init__:479 - Creating TimeData with data type float32
2023-04-28 08:06:36.946 | INFO     | resistics.time:run:1607 - Removing mean from channels ['Hx', 'Hy', 'Hz', 'Ex', 'Ey']
2023-04-28 08:06:36.953 | DEBUG    | resistics.time:__init__:479 - Creating TimeData with data type float32


In [23]:
dec_params = config.dec_setup.run(time_data.metadata.fs)
dec_data = config.decimator.run(dec_params, time_data)

2023-04-28 08:06:37.003 | INFO     | resistics.decimate:run:759 - Decimating level 0 with factor 1
2023-04-28 08:06:37.004 | INFO     | resistics.decimate:run:759 - Decimating level 1 with factor 4
2023-04-28 08:06:37.005 | INFO     | resistics.time:run:2238 - Decimating by 4 in 2 step(s) with factors [2, 2]
2023-04-28 08:06:37.053 | DEBUG    | resistics.time:__init__:479 - Creating TimeData with data type float32
2023-04-28 08:06:37.054 | INFO     | resistics.decimate:run:759 - Decimating level 2 with factor 8
2023-04-28 08:06:37.054 | INFO     | resistics.time:run:2238 - Decimating by 8 in 3 step(s) with factors [2, 2, 2]
2023-04-28 08:06:37.066 | DEBUG    | resistics.time:__init__:479 - Creating TimeData with data type float32
2023-04-28 08:06:37.068 | INFO     | resistics.decimate:run:759 - Decimating level 3 with factor 4
2023-04-28 08:06:37.068 | INFO     | resistics.time:run:2238 - Decimating by 4 in 2 step(s) with factors [2, 2]
2023-04-28 08:06:37.071 | DEBUG    | resistics.ti

In [24]:
win_params = config.win_setup.run(dec_data.metadata.n_levels, dec_data.metadata.fs)
win_data = config.windower.run(ref_time, win_params, dec_data)

2023-04-28 08:06:37.106 | INFO     | resistics.window:run:1330 - Windowing decimation level 0
2023-04-28 08:06:37.107 | DEBUG    | resistics.window:get_first_and_last_win:485 - Adjusting last window attempt 1
2023-04-28 08:06:37.110 | INFO     | resistics.window:run:1336 - 3765 windows, size 128, overlap 32
2023-04-28 08:06:37.112 | INFO     | resistics.window:run:1330 - Windowing decimation level 1
2023-04-28 08:06:37.113 | DEBUG    | resistics.window:get_first_and_last_win:485 - Adjusting last window attempt 1
2023-04-28 08:06:37.115 | INFO     | resistics.window:run:1336 - 941 windows, size 128, overlap 32
2023-04-28 08:06:37.116 | INFO     | resistics.window:run:1330 - Windowing decimation level 2
2023-04-28 08:06:37.117 | DEBUG    | resistics.window:get_first_and_last_win:485 - Adjusting last window attempt 1
2023-04-28 08:06:37.118 | INFO     | resistics.window:run:1336 - 117 windows, size 128, overlap 32
2023-04-28 08:06:37.120 | INFO     | resistics.window:run:1330 - Windowing 

In [25]:
spec_data = config.fourier.run(win_data)
eval_data = config.evals.run(dec_params, spec_data)

2023-04-28 08:06:37.150 | INFO     | resistics.spectra:run:487 - Performing fourier transforms of windowed decimated data
2023-04-28 08:06:37.151 | INFO     | resistics.spectra:run:489 - Transforming level 0
2023-04-28 08:06:37.152 | DEBUG    | resistics.spectra:run:493 - Padding size 128 to next fast len 128
2023-04-28 08:06:37.209 | INFO     | resistics.spectra:run:489 - Transforming level 1
2023-04-28 08:06:37.209 | DEBUG    | resistics.spectra:run:493 - Padding size 128 to next fast len 128
2023-04-28 08:06:37.221 | INFO     | resistics.spectra:run:489 - Transforming level 2
2023-04-28 08:06:37.222 | DEBUG    | resistics.spectra:run:493 - Padding size 128 to next fast len 128
2023-04-28 08:06:37.225 | INFO     | resistics.spectra:run:489 - Transforming level 3
2023-04-28 08:06:37.225 | DEBUG    | resistics.spectra:run:493 - Padding size 128 to next fast len 128
2023-04-28 08:06:37.228 | INFO     | resistics.spectra:run:504 - Fourier transforms completed
2023-04-28 08:06:37.229 | DE

In [26]:
gathered_data = QuickGather().run(dir_path, dec_params, config.tf, eval_data)
reg_data = config.regression_preparer.run(config.tf, gathered_data)

2023-04-28 08:06:37.268 | INFO     | resistics.gather:run:847 - Quick gathering data for regression prepartion
2023-04-28 08:06:37.270 | INFO     | resistics.regression:run:388 - Preparing regression data
2023-04-28 08:06:37.271 | INFO     | resistics.regression:run:389 - Out chans site: kap123
2023-04-28 08:06:37.271 | INFO     | resistics.regression:run:390 - Out chans: ['Ex', 'Ey']
2023-04-28 08:06:37.272 | INFO     | resistics.regression:run:391 - In chans site: kap123
2023-04-28 08:06:37.272 | INFO     | resistics.regression:run:392 - In chans: ['Hx', 'Hy']
2023-04-28 08:06:37.273 | INFO     | resistics.regression:run:393 - Cross chans site: kap123
2023-04-28 08:06:37.273 | INFO     | resistics.regression:run:394 - Cross chans: ['Hx', 'Hy']
2023-04-28 08:06:37.277 | INFO     | resistics.regression:_get_regression_data:419 - Preparing regression data for 20 frequencies
100%|██████████| 20/20 [00:00<00:00, 183.73it/s]


In [27]:
soln = config.solver.run(reg_data)
fig = soln.tf.plot(
    soln.freqs,
    soln.components,
    to_plot=["ExHy", "EyHx"],
    x_lim=[1, 5],
    res_lim=[0, 4],
    legend="128",
    symbol="circle",
)
fig.update_layout(height=900)
fig.show()

2023-04-28 08:06:37.410 | INFO     | resistics.regression:_solve:782 - Solving for 20 evaluation frequencies
100%|██████████| 20/20 [00:04<00:00,  4.50it/s]
